In [743]:
#Digits
# from sklearn import datasets
# from sklearn import metrics
# from sklearn.neighbors import NearestNeighbors
# import numpy as np
# import random
# iris = datasets.load_iris()
# digits = datasets.load_digits()
# news = datasets.fetch_20newsgroups
# news_train = news(subset='train', categories= ['comp.os.ms-windows.misc', 'comp.sys.mac.hardware'])
# data = digits.data
# actual_target = np.zeros(len(digits.target))
# for i in range(len(digits.target)):
#     if digits.target[i] <= 4:
#         actual_target[i] = 0.
#     if digits.target[i] > 4:
#         actual_target[i] = 1.
# print(actual_target[0:30])
# target = actual_target.copy()
# for i in range(len(digits.target)):
#     #if digits.target[i] == 2:
#     #    target[i] = 1.
#     if (i%3) == 0:
#         target[i] = 0.5
# print(target[0:30])

In [899]:
#titanic
from sklearn import datasets
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
import pandas as pd 
data_train = pd.read_csv('titanic_data_train.csv', sep=',')
target_train = pd.read_csv('titanic_labels_train.csv', sep=',')
iris = datasets.load_iris()
digits = datasets.load_digits()
news = datasets.fetch_20newsgroups
news_train = news(subset='train', categories= ['comp.os.ms-windows.misc', 'comp.sys.mac.hardware'])
data = np.array(data_train)
target = np.array(target_train)
print(data.shape)
actual_target = target.copy()
for i in range(len(target)):
    #if digits.target[i] == 2:
    #    target[i] = 1.
    if (i%2) == 0:
        target[i] = 0.5
print(target.shape)

(890, 5)
(890, 1)


In [900]:
#calculate pairwise distance #https://papers.nips.cc/paper/1896-data-clustering-by-markovian-relaxation-and-the-information-bottleneck-method.pdf
def distance(data):
    distance_matrix = metrics.pairwise_distances(data)
    return distance_matrix

In [901]:
#find indices of nearest neighbors
def knearest(data,k):
    neighbors = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(data)
    distances, indices = neighbors.kneighbors(data)
    return neighbors.kneighbors(data)

In [902]:
#Assign weights to nearest neighbors
def weights(data,d,indices,sigma):
    weights = np.zeros((data.shape[0],data.shape[0]))
    for i in range(data.shape[0]):
        for j in range(indices.shape[1]):
            weights[indices[i][j],i]=np.exp(-d[indices[i][j],i]/sigma**2)
            weights[i,i]=1
    return weights

In [903]:
#one step transition matrix
def p_one(weights):
    num = np.zeros((weights.shape))
    denom = np.zeros(weights.shape[0])
    p = np.zeros((weights.shape))
    for i in range(p.shape[0]):
        denom[i] = np.sum(weights[i,:])
        for j in range(p.shape[0]):
            num[i,j] = weights[i,j]
            if denom[i] != 0:
                p[i,j] = num[i,j]/denom[i]
    return p

In [904]:
#t-step transition matrix
def p_t(p_one_m,t):
    return np.linalg.matrix_power(p_one_m,10)

In [905]:
#E-step
def E_step(p_t_matrix,P_y):
    return p_t_matrix*P_y

In [906]:
#M-step
def M_step(e_step,p_t_matrix):
    denom = np.zeros(p_t_matrix.shape[1])
    num = np.zeros(e_step.shape[1])
    for i in range(p_t_matrix.shape[1]):
        denom[i] = np.sum(p_t_matrix[:,i])
        num[i] = np.sum(e_step[:,i])
    P_y=num/denom
    return P_y

In [907]:
#Loglikelihood
def likelihood(p_t_matrix,P_y):
    N_sum = np.zeros(p_t_matrix.shape[1])
    #print(p_t_matrix.shape[1])
    for i in range(p_t_matrix.shape[1]):
        N_sum[i] = np.sum(p_t_matrix[:,i]*P_y[:])
    log_N_sum = np.sum(np.log(N_sum))
    return log_N_sum  

In [908]:
#pairwise distance
d = distance(data)
#indices from k-nearest neighbor
distances, indices = knearest(data,5)
#find weights
Weights = weights(data,d,indices,1)
#one step transition probability matrix
p_one_matrix = p_one(Weights)
#t step transition probability matrix
p_t_matrix = p_t(p_one_matrix,2)

P_y=np.zeros((data.shape[0],1))
print(np.sum(target))
print(target.shape)
for i in range(len(target)):
    P_y[i] = target[i]/1

print(P_y)
likelihood_new = 1
likelihood_old = 0
#iterate until loglikelihood no longer improves
iteration = 0
max_iter = 100
#while (abs(likelihood_new-likelihood_old) > .0003):
for i in range(max_iter):
    iteration += 1
    print("iteration = ",iteration)
    likelihood_old = likelihood_new
    e_step=E_step(p_t_matrix,P_y)
    print("finish e")
    P_y = M_step(e_step,p_t_matrix)
    print("P = ",P_y)
    print("finish m")
    #likelihood_new = likelihood(p_t_matrix,P)
    print("finish likelihood")

predictions = np.zeros(data.shape[0])
#make classification predictions
for i in range(data.shape[0]):
    if P_y[i] >= 0.5:
        predictions[i] = 1
    if P_y[i] < 0.5:
        predictions[i] = 0
print(predictions)
print(np.sum(predictions))

380.5
(890, 1)
[[0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5]
 [1. ]
 [0.5]
 [0. ]
 [0.5]
 [0. ]
 [0.5

 5.04080932e-01 1.99514208e-01]
finish m
finish likelihood
iteration =  6
finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-

P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-01
 6.74547004e-01 6.85039488e-02 3.49437345e-01 2.17322741e-01
 4.92876229e-01 4.7

 5.04080932e-01 1.99514208e-01]
finish m
finish likelihood
iteration =  15
finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e

finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-01
 6.74547004e-01 6.85039488e-02 3.49437345e-01 2.17322741e-01
 4.9287622

...

finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-01
 6.74547004e-01 6.85039488e-02 3.49437345e-01 2.17322741e-01
 4.9287622

P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-01
 6.74547004e-01 6.85039488e-02 3.49437345e-01 2.17322741e-01
 4.92876229e-01 4.7

 5.04080932e-01 1.99514208e-01]
finish m
finish likelihood
iteration =  88
finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e

 5.04080932e-01 1.99514208e-01]
finish m
finish likelihood
iteration =  92
finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e

iteration =  97
finish e
P =  [6.57443593e-01 3.34387660e-01 5.81503434e-01 1.88198523e-01
 3.91138038e-01 2.28064420e-01 5.83235696e-01 6.31783213e-01
 4.28953891e-01 7.01761086e-01 3.51766156e-01 2.54547827e-01
 3.02922561e-01 5.94948899e-01 3.36465126e-01 2.93043192e-01
 2.87155320e-01 4.59740120e-01 1.95435866e-01 3.62883630e-01
 4.17953227e-01 6.08006263e-01 4.60644418e-01 1.88182439e-01
 3.03113532e-01 6.32828457e-02 6.48850632e-01 8.05555940e-01
 4.44444554e-01 3.47190012e-01 4.97166765e-01 6.61968300e-01
 3.68601044e-01 4.69692810e-01 5.06384048e-01 7.39479202e-01
 4.14655921e-01 3.02326469e-01 5.41535188e-01 4.04955926e-01
 6.00983147e-01 8.33331687e-02 6.48223934e-01 4.47570049e-01
 4.33224353e-01 4.87078973e-01 4.60039254e-01 4.01967849e-01
 2.99869768e-01 1.64045305e-01 3.96570619e-01 6.06743449e-01
 5.30980465e-01 2.25324646e-04 4.63460427e-01 4.85427334e-01
 4.19429814e-01 7.51981691e-01 2.55170531e-01 2.96503914e-01
 6.74547004e-01 6.85039488e-02 3.49437345e-01 2.1732274

In [909]:
predictions = np.zeros(data.shape[0])
#make classification predictions
for i in range(data.shape[0]):
    if P_y[i] >= 0.5:
        predictions[i] = 1
    if P_y[i] < 0.5:
        predictions[i] = 0
print(metrics.accuracy_score(actual_target, predictions))
print(predictions[0:30])

0.7685393258426966
[1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0.]
